In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from bs4 import BeautifulSoup
import csv

In [4]:
##Step 1: Login to Linkedin 
# Open Chrome and login linkedin login site
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()
url = "https://www.linkedin.com/login"
driver.get(url)
sleep(2)

# Import username and password
credential = open('login_credential.txt')
line = credential.readlines()
username = line[0]
password = line[1]

# key in username/password --> login
driver.find_element("xpath","//input[@id = 'username']").send_keys(username)
sleep(2)
driver.find_element("xpath","//input[@id = 'password' ]").send_keys(password)
sleep(2)
driver.find_element("xpath","//button[@aria-label= 'Sign in' ]").click()
sleep(2)

## Step 2: Search for the profile we want to crawl
search_field =  driver.find_element("xpath", "//input[@aria-label = 'Search']")
search_field.send_keys("Data Analyst")
search_field.send_keys(Keys.RETURN)
sleep(10)
driver.find_element("xpath","//nav[@aria-label = 'Search filters']//button[text()= 'People']").click()
sleep(5)

## Step 3: Scrape the URLs of the profiles
def  GetURL():
    page_source = BeautifulSoup(driver.page_source)
    profiles =  page_source.find_all('a', class_= 'app-aware-link scale-down')
    all_profile_URL = []
    for profile in profiles:
        profile_ID = profile.get('href')
        all_profile_URL.append(profile_ID)
    return all_profile_URL


input_page = int(input("How many pages you want to scrape: "))
URLs_all_page = []
for page in range(input_page):
    URLs_one_page = GetURL()
    sleep(5)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    sleep(2)
    next_button =  driver.find_element("xpath", "//button[@aria-label = 'Next']")
    sleep(2)
    driver.execute_script("arguments[0].click();", next_button)
    URLs_all_page = URLs_all_page + URLs_one_page
    sleep(2)


print(URLs_all_page)
  

C:\Users\Admin\AppData\Local\Temp\ipykernel_7500\3580544358.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


['https://www.linkedin.com/in/lehuyhoanguon?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC6WPsIBY923-AfwEFJWo2a3jDykGN4gROg', 'https://www.linkedin.com/in/hoangtong27?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADoeNqYBwIwIQWEldQtcmg8rhG97CKSAQH4', 'https://www.linkedin.com/in/hang-dao-2661b516a?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAChAii4Byz3ZeIVvBMu5eP54gO8yWdTo-ew', 'https://www.linkedin.com/in/hainguyen88?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABYB-gsB3lzs1hRbjmNhPEvm-C89brnG8qw', 'https://www.linkedin.com/in/trungbuiq?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC70my8BV4J-aPz9RwZKBLy_UNk_ZEG4MOY', 'https://www.linkedin.com/in/l%C3%A2n-l%C3%AA-85381121b?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADd4rH8BtiD_-1z0JOZ_kTB-PKBrYtE0h14', 'https://www.linkedin.com/in/linh-nguyen-74217823a?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADt4HSIBb4g1EjEVWYV_Yvuye1bEsoW980s', 'https://www.linkedin.com/in/truongsonla?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AA

In [5]:
## Step 4: Scrape the data of 1 linkedin profile, and write the data to a .CSV file
with open('output.csv', 'w', newline= '')as file_output: 
    headers = ['Name', 'Job Title', 'Location', 'URL']
    writer = csv.DictWriter(file_output, delimiter = ',', lineterminator = '\n', fieldnames = headers)
    writer.writeheader() 

    for linkedin_URL in URLs_all_page:
        driver.get(linkedin_URL)
        sleep(5)
        page_source = BeautifulSoup(driver.page_source, "html.parser")
        infor_div = page_source.find('div', class_="mt2 relative")
        infor_loc = infor_div.find_all('div', class_ = 'pv-text-details__left-panel')
        name = infor_loc[0].find('h1').get_text()
        print('Profile name is: ',name)
        title = infor_loc[0].find('div', class_ = 'text-body-medium break-words').get_text().strip()
        print('Profile title is: ',title)
        location = infor_loc[1].find('span').get_text().strip()     
        print('Profile location is: ',location)
        writer.writerow({headers[0]:name,headers[1]:title, headers[2]: location, headers[3]: linkedin_URL}) 
        print('\n')

    

Profile name is:  Hoàng Lê
Profile title is:  Data Analyst at Honda Vietnam Co., Ltd.
Profile location is:  Hanoi, Hanoi, Vietnam


Profile name is:  Hoang Tong
Profile title is:  Data Analyst
Profile location is:  Hanoi, Hanoi, Vietnam


Profile name is:  Hang Dao
Profile title is:  Data analyst
Profile location is:  Hanoi Capital Region


Profile name is:  Hai Nguyen
Profile title is:  Data Analyst
Profile location is:  Vietnam


Profile name is:  Trung Bui
Profile title is:  Data Analyst at Viettel Solutions Corporation
Profile location is:  Hanoi, Hanoi, Vietnam


Profile name is:  Lân Lê
Profile title is:  Data Analyst at Viettel Digital
Profile location is:  Hanoi, Hanoi, Vietnam


Profile name is:  Linh Nguyen
Profile title is:  Data Analyst
Profile location is:  Hanoi Capital Region


Profile name is:  Truong Son La
Profile title is:  Data Analyst
Profile location is:  Hanoi Capital Region


Profile name is:  Tùng Trần
Profile title is:  Associate Data Analyst
Profile location 